# Machine Learning Workflow
Complete the steps below to complete the machine learning workflow for this classifier.

In [1]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\N12667\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\N12667\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import re
import pandas as pd
import numpy as np

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, accuracy_score

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def load_data():
    df = pd.read_csv('corporate_messaging.csv', encoding='latin-1')
    df = df[(df["category:confidence"] == 1) & (df['category'] != 'Exclude')]
    X = df.text.values
    y = df.category.values
    return X, y

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    #text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # added by me
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [4]:
# convert sparse matrix to numpy array to view
import pandas as pd

def vect_to_df(X, vect):
    keys = list(vect.vocabulary_.keys())
    values = vect.vocabulary_.values()
    mapper = {keys[i]: vect.vocabulary_[keys[i]] for i in values} 
    cols = [key for key, value in sorted(mapper.items(), key=lambda kv: kv[1])] # sort by value
    return X#pd.DataFrame(X.toarray(), columns=cols)

### Step 1: Load data and perform a train test split

In [5]:
# load data
X, y = load_data()

# perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Step 2: Train classifier
* Fit and transform the training data with `CountVectorizer`. Hint: You can include your tokenize function in the `tokenizer` keyword argument!
* Fit and transform these word counts with `TfidfTransformer`.
* Fit a classifier to these tfidf values.

In [6]:
# Instantiate transformers and classifier
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer(smooth_idf=False)
clf = MultinomialNB()

# Fit and/or transform each to the data
X_train = vect.fit_transform(X_train)
X_train = tfidf.fit_transform(X_train)
clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### Step 3: Predict on test data
* Transform (no fitting) the test data with the same CountVectorizer and TfidfTransformer
* Predict labels on these tfidf values.

In [7]:
# Transform test data
X_test = vect.transform(X_test)
X_test = tfidf.transform(X_test)

# Predict test labels
y_pred = clf.predict(X_test)

### Step 4: Display results
Display a confusion matrix and accuracy score based on the model's predictions.

# Final Step: Refactor
Organize these steps into the following functions.

In [39]:
from sklearn.ensemble import RandomForestClassifier

def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels) 
    accuracy = accuracy_score(y_test, y_pred)

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)


def main():
    # load data
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)#, random_state=42)
    
    # Instantiate transformers and classifier
    vect = CountVectorizer(tokenizer=tokenize)
    tfidf = TfidfTransformer()
    clf = RandomForestClassifier(n_estimators=100) #MultinomialNB()

    # Fit and/or transform each to the data
    X_train = vect.fit_transform(X_train)
    X_train = tfidf.fit_transform(X_train)
    clf.fit(X_train, y_train)
    
    # Transform test data
    X_test = vect.transform(X_test)
    X_test = tfidf.transform(X_test)

    # Predict test labels
    y_pred = clf.predict(X_test)

    display_results(y_test, y_pred)


In [40]:
# run program
main()

Labels: ['Action' 'Dialogue' 'Information']
Confusion Matrix:
 [[ 86   0  27]
 [  0  22   3]
 [  3   1 459]]
Accuracy: 0.9434276206322796
